In [ ]:
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem import rdFingerprintGenerator

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, roc_auc_score
from sklearn.utils import shuffle

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

from joblib import dump

from rdkit import RDLogger

# Imposta il livello di log a ERROR (ignora INFO e WARNING)
lg = RDLogger.logger()
lg.setLevel(RDLogger.ERROR)

# 1) Leggi dataset, mappa -1->0, +1->1, 0->NaN
df = pd.read_csv("../data/data_train.csv")
task_cols = [f"task{i}" for i in range(1,12)]

for c in task_cols:
    df[c] = df[c].map({
        -1: 0,
        1: 1,
        0: np.nan
    })

import numpy as np
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, MACCSkeys
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem.MolStandardize import rdMolStandardize

def standardize_mol(mol):
    """
    Esempio minimal di standardizzazione:
    1) Cleanup chimico di base
    2) Tieni solo il frammento più grande
    (Volendo puoi aggiungere tautomer enumeration, neutralizzazione, etc.)
    """
    # 1) Pulizia di base
    mol = rdMolStandardize.Cleanup(mol)
    # 2) Tieni solo il frammento principale
    lfc = rdMolStandardize.LargestFragmentChooser()
    mol = lfc.choose(mol)
    return mol

def smiles_to_fp(smiles, 
                 nBits=1024, 
                 radius=2, 
                 use_MACCS=False, 
                 standardize=True):
    """
    Converte uno SMILES in fingerprint Morgan (di default).
    Se use_MACCS=True, concatena anche il fingerprint MACCS
    Se standardize=True, prova a "ripulire" la molecola prima di calcolare il FP
    """

    # Converte da SMILES a Mol
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        # Se la conversione fallisce, restituisci un vettore di zeri
        # con dimensione pari a nBits (Morgan) o nBits + 166 se concateni MACCS
        maccs_size = 166 if use_MACCS else 0
        return np.zeros(nBits + maccs_size, dtype=np.uint8)

    # Standardizzazione (opzionale)
    if standardize:
        mol = standardize_mol(mol)

    # Genera fingerprint Morgan (usando rdFingerprintGenerator o AllChem)
    fpgen = rdFingerprintGenerator.GetMorganGenerator(radius=radius, fpSize=nBits)
    fp_morgan = fpgen.GetFingerprint(mol)
    arr_morgan = np.zeros((nBits,), dtype=np.uint8)
    DataStructs.ConvertToNumpyArray(fp_morgan, arr_morgan)

    if not use_MACCS:
        # Ritorna solo Morgan
        return arr_morgan

    # Altrimenti, calcola MACCS e concatena
    maccs_fp = MACCSkeys.GenMACCSKeys(mol)
    arr_maccs = np.zeros((maccs_fp.GetNumBits(),), dtype=np.uint8)
    DataStructs.ConvertToNumpyArray(maccs_fp, arr_maccs)

    # Concatenazione Morgan + MACCS
    combined_fp = np.concatenate([arr_morgan, arr_maccs])
    return combined_fp


# 3) Costruisci fingerprint per tutti
X = np.array([smiles_to_fp(s, 
                           nBits=1024, 
                           radius=2, 
                           use_MACCS=True, 
                           standardize=True) 
              for s in df["smiles"]],
             dtype=np.float32)

# 4) Unico split train/test
df_train, df_test, X_train, X_test = train_test_split(df, X, test_size=0.05, random_state=42)

# Parametri di esempio per la ricerca
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [None, 5, 10]
#     # puoi aggiungere 'min_samples_split': [2, 5] ecc.
# }

# param_grid = {
#     'n_estimators': [5, 10, 20, 50, 100, 200, 300, 500, 700, 1000],  # Numero di alberi nella foresta
#     'criterion': ['gini', 'entropy'],         # Funzione per misurare la qualità di uno split (per classificazione)
#                                               # Se fai regressione, usa: ['squared_error', 'absolute_error']
#     'max_depth': [None, 1, 3, 5, 10, 20, 22, 27, 30],       # Massima profondità di ciascun albero (None = nodi espansi finché puri o min_samples_split)
#     'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],          # Numero minimo di campioni richiesti per splittare un nodo interno
#     'min_samples_leaf': [1, 3, 5, 6, 7],           # Numero minimo di campioni richiesti in un nodo foglia
#     'max_features': ['sqrt', 'log2', 0.5, 0.7],
#     'class_weight': [None, 'balanced'] 
# }

search_spaces = {
    'n_estimators': Integer(5, 2500),  # Range intero (limiti inclusi)
    'criterion': Categorical(['gini', 'entropy']), # Valori discreti
    'max_depth': Categorical([None, 5, 6, 7, 8, 9, 10, 11, 13, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35]), # Valori discreti, incluso None e range affinato
    'min_samples_split': Integer(2, 15), # Range intero
    'min_samples_leaf': Integer(1, 10),  # Range intero
    'max_features': Categorical(['sqrt', 'log2', 0.5, 0.7]), # Valori discreti (manteniamo questo semplice per ora)
    # Nota: Si potrebbe usare Real(0.1, 1.0) per max_features, ma gestire misto float/string è più complesso.
    # Mantenere Categorical con i valori che hanno funzionato è ragionevole.
    'class_weight': Categorical([None, 'balanced']) # Valori discreti
}

auc_list = []

for i, task in enumerate(task_cols[4:5], start=5):
    print(f"\n*** Tuning per {task} ***")
    # a) Seleziona i sample definiti per training
    train_mask = ~df_train[task].isna()
    X_train_f = X_train[train_mask]
    y_train_f = df_train.loc[train_mask, task].values.astype(int)

    if len(y_train_f) == 0:
        print(f"Nessun sample train per {task}, skip.")
        continue

    # b) Esegui una Grid Search
    rf = RandomForestClassifier(random_state=0)  
    # grid_search = GridSearchCV(
    #     estimator=rf,
    #     param_grid=param_grid,
    #     scoring='roc_auc', # ottimizziamo AUC
    #     cv=3,              # 3-fold cross validation
    #     n_jobs=-1
    # )

    # grid_search = RandomizedSearchCV(
    #     estimator=rf,
    #     param_distributions=param_grid, # Nota: per RandomizedSearchCV si chiama param_distributions
    #     n_iter=1000,                     # <<< AGGIUNGI QUESTO! Numero di combinazioni da provare
    #     scoring='roc_auc',
    #     cv=5,
    #     n_jobs=-1,
    #     random_state=42 # Aggiungi per riproducibilità della ricerca casuale
    # )
    grid_search = BayesSearchCV(
        estimator=rf,
        search_spaces=search_spaces, # Usa gli spazi definiti sopra
        n_iter=100,       # Numero di combinazioni da provare
        scoring='roc_auc',
        cv=5,                      # CONSIGLIATO: 5-fold CV
        n_jobs=-1,
        random_state=42            # Per riproducibilità della ricerca
    )
    grid_search.fit(X_train_f, y_train_f)

    best_model = grid_search.best_estimator_
    print(f"Best params for {task}: {grid_search.best_params_}")

    # Salvataggio del modello
    dump(best_model, f"rf_task{i}.joblib")
    print(f"Salvato: rf_task{i}.joblib")

    # c) Valutazione su test
    test_mask = ~df_test[task].isna()
    X_test_f = X_test[test_mask]
    y_test_f = df_test.loc[test_mask, task].values.astype(int)

    if len(y_test_f) == 0:
        print(f"Nessun sample test per {task}, skip.")
        continue

    # Predici prob
    y_proba = best_model.predict_proba(X_test_f)[:, 1]  # output continuo
    y_pred = (y_proba >= 0.5).astype(int)

    precision = precision_score(y_test_f, y_pred, zero_division=0)
    if len(np.unique(y_test_f)) == 2:
        auc_val = roc_auc_score(y_test_f, y_proba)
    else:
        auc_val = np.nan

    print(f"{task} -> Precision={precision:.3f}, AUC={auc_val if not np.isnan(auc_val) else 'N/A'}")
    auc_list.append(auc_val)

valid_aucs = [x for x in auc_list if not np.isnan(x)]
mean_auc = np.mean(valid_aucs) if valid_aucs else np.nan
print(f"\nAUC Media sui task: {mean_auc:.3f}")



*** Tuning per task5 ***
